### Reimplementation of Image Super-Resolution Using Deep Convolutional Network in PyTorch

This notebook is the reimplementation of this [paper](https://arxiv.org/abs/1501.00092)

In [1]:
import h5py
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
torch.manual_seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Load dataset

In [2]:
class MyDataset(Dataset):
    def __init__(self, file_path):
        self.file = h5py.File(file_path, 'r')
        self.data = self.file['data']
        self.label = self.file['label']
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,idx):
        image = torch.tensor(self.data[idx], dtype=torch.float32)
        label = torch.tensor(self.label[idx], dtype=torch.float32)
        
        return image, label
    
    def close(self):
        self.file.close()       

### Preparing Datasets and Dataloaders

The train.h5 and test_set.h5 files are obtained by running the MATLAB scripts generate_train and generate_test respectively from [this](https://mmlab.ie.cuhk.edu.hk/projects/SRCNN/SRCNN_train.zip) source code given in the original paper

In [3]:
# Define dataset and dataloader
dataset = MyDataset('train.h5') 
train_loader = DataLoader(dataset, batch_size=128)

dataset_test_set5 = MyDataset('test.h5')
val_loader = DataLoader(dataset_test_set5, batch_size=1)

### Defining the CNN model

In [6]:
# Define the SRCNN model
class SRCNN(nn.Module):
    def __init__(self):
        super(SRCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=9, stride=1, padding=0)  # Changed input channels to 1
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(64, 32, kernel_size=1, stride=1, padding=0)
        self.relu2 = nn.ReLU()
        self.conv3 = nn.Conv2d(32, 1, kernel_size=5, stride=1, padding=0)  # Changed output channels to 1
        
        self._initialize_weights()

    def forward(self, x):
        out = self.relu1(self.conv1(x))
        out = self.relu2(self.conv2(out))
        out = self.conv3(out)
        # Resize output to match labels dimensions (21, 21)
        #out = F.interpolate(out, size=(21, 21), mode='bicubic', antialias=False)
        return out
    
    def _initialize_weights(self):
        # Initialize weights with Gaussian distribution (std=0.001) and bias with zeros
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                # Initialize convolutional layer weights with Gaussian distribution (std=0.001)
                nn.init.normal_(m.weight, mean=0, std=0.001)
                if m.bias is not None:
                    # Initialize bias with zeros
                    nn.init.constant_(m.bias, 0)

model = SRCNN()

### Defining hyperparameters

In [7]:

# Define different learning rates and multipliers for weights and biases
learning_rates = {
    'conv1_weight': 1e-4 * 1.0,
    'conv1_bias': 1e-4  * 0.1,
    'conv2_weight': 1e-4 * 1.0,
    'conv2_bias': 1e-4 * 0.1,
    'conv3_weight': 1e-5 * 0.1,
    'conv3_bias': 1e-5 * 0.1 
}

# Group parameters and assign specific learning rates with multipliers
param_groups = []
for name, param in model.named_parameters():
    if name in learning_rates:
        lr = learning_rates[name]
    else:
        lr = 0.0001  # Default learning rate for other parameters

    param_groups.append({'params': param, 'lr': lr})

optimizer = optim.SGD(param_groups, lr=0.0001, momentum=0.9, weight_decay=1e-4)

criterion = nn.MSELoss()
num_epochs = 10

### Training loop

In [8]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for batch_idx, (inputs, labels) in enumerate(train_loader):
        # Forward pass
        outputs = model(inputs)
        # Print shapes of outputs and labels
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Print average loss after each epoch
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}")

# Save the trained model
torch.save(model.state_dict(), 'srcnn_model.pth')

Epoch 1, Loss: 0.19692319421207202
Epoch 2, Loss: 0.11591762593563865
Epoch 3, Loss: 0.07518237704301582
Epoch 4, Loss: 0.05482223765815006
Epoch 5, Loss: 0.04464743910905193
Epoch 6, Loss: 0.03956396030809949
Epoch 7, Loss: 0.03702495014842819
Epoch 8, Loss: 0.035757218180772134
Epoch 9, Loss: 0.03512446486993748
Epoch 10, Loss: 0.034808737751753895


In [9]:
#Evaluate model on validation dataset
model.eval()
val_loss = 0.0
for batch_idx, (val_inputs, val_labels) in enumerate(val_loader):
    val_outputs = model(val_inputs)
    val_loss += criterion(val_outputs, val_labels).item()

average_val_loss = val_loss / len(val_loader)
print(f"Validation Loss: {average_val_loss}")

Validation Loss: 0.05704200948140895


### Calculating PSNR values for validation images

In [10]:
# Function to calculate PSNR between two images
def calculate_psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')  # PSNR is infinite if images are identical
    MAX = 255.0
    psnr = 10 * np.log10((MAX**2) / mse)
    return psnr

In [11]:
psnr_values = []

for inputs, labels in val_loader:
    # Move inputs and labels to the device
    inputs = inputs.to(device)
    labels = labels.to(device)
    
    # Generate predictions using the model
    with torch.no_grad():
        outputs = model(inputs)
    
    outputs = outputs.cpu().detach().numpy()
    labels = labels.cpu().detach().numpy()
    
    # Calculate PSNR for each image pair
    for i in range(outputs.shape[0]):  # Iterate over batch size
        psnr_value = calculate_psnr(outputs[i].squeeze(), labels[i].squeeze())
        psnr_values.append(psnr_value)

# Compute average PSNR over all images in the dataset
average_psnr = np.mean(psnr_values)
print(f"Average PSNR: {average_psnr:.2f} dB")

Average PSNR: 62.23 dB
